<a href="https://colab.research.google.com/github/arkeyan/Indicators/blob/main/Kompozit_Breakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed  matplotlib openpyxl
!pip install tradingview-screener==2.5.0

import numpy as np
import pandas as pd
from scipy.signal import argrelextrema
from tradingview_screener import get_all_symbols
from tvDatafeed import TvDatafeed, Interval
from scipy.stats import linregress

# Normalizasyonu XU100'e göre yapma
def normalize_data(hisse_data, xu100_data):
    columns_to_normalize = ['open', 'high', 'low', 'close', 'volume']
    result = hisse_data.copy()
    for column in columns_to_normalize:
        result[column] = 100 * (hisse_data[column] / xu100_data[column])  # XU100 ile normalize et
    return result

# Down trend çizgisi analizi
def Down_Trend_Line(data, window=5):
    df = data.iloc[:-window].copy()
    hh_pairs = argrelextrema(df['high'].values, comparator=np.greater, order=window)[0]
    highest_point = df.iloc[hh_pairs].nlargest(1, 'high').sort_values(by='datetime')
    max_idx = data.index.get_loc(highest_point.index[0])
    df_next = df.iloc[max_idx:]
    hh_pairs_next = argrelextrema(df_next['high'].values, comparator=np.greater, order=window)[0]
    slopes = []
    intercepts = []

    for hh_next in hh_pairs_next:
        next_hp = df_next.iloc[hh_next]
        slope, intercept, _, _, _ = linregress([highest_point.index[0], next_hp.name], [highest_point['high'].values[0], next_hp['high']])
        slopes.append(slope)
        intercepts.append(intercept)

    slopes = [slope for slope in slopes if slope != 0]
    intercepts = [intercept for slope, intercept in zip(slopes, intercepts) if slope != 0]
    if slopes:
        min_slope_index = np.argmin(np.abs(slopes))
        min_slope = slopes[min_slope_index]
        min_intercept = intercepts[min_slope_index]
        df = data.copy()
        df['trend'] = min_slope * df.index + min_intercept
        df['Entry'] = df['high'] > df['trend']
    return df

tv = TvDatafeed()

# Kullanıcıdan zaman dilimi seçimi
def get_interval_choice():
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("\nLütfen bir zaman dilimi seçin (rakam girilecek):")
    for key, value in intervals.items():
        print(f"{key}: {value[0]}")

    choice = input("Zaman dilimini seçin (1-9): ")
    interval_name, interval = intervals.get(choice, ('4 Saat', Interval.in_4_hour))  # Default to 4-hour if invalid choice
    return interval_name, interval


# Hisseler ve zaman dilimi seçimi
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

interval_name, interval = get_interval_choice()  # Kullanıcıdan zaman dilimi seçimini alıyoruz

Titles = ['Hisse Adı', 'Son Fiyat', 'Trend Değeri', 'Yüzde', 'Yakınlık Durumu', 'Kırılma Durumu']
df_down_trend = pd.DataFrame(columns=Titles)

# XU100 verisini çekme
xu100_data = tv.get_hist(symbol='XU100', exchange='BIST', interval=interval, n_bars=500)
xu100_data = xu100_data.reset_index()

for i in range(0, len(Hisseler)):
    try:
        order = 5
        data = tv.get_hist(symbol=Hisseler[i], exchange='BIST', interval=interval, n_bars=500)
        data = data.reset_index()

        # XU100 ile normalize et
        normalized_data = normalize_data(data, xu100_data)

        # Down trend çizgisi analizi
        Trend_line = Down_Trend_Line(normalized_data, window=order)
        Entry = False
        Signals = Trend_line.tail(order)
        Signals = Signals.reset_index()

        Last_Price = Signals.loc[order-1, 'high']
        Last_Trend = Signals.loc[order-1, 'trend']
        Last_Perc = ((Signals.loc[order-1,'trend'] - Signals.loc[order-1,'high']) / Signals.loc[order-1,'trend']) * 100
        Close_Status = False
        Break_Status = False

        if Last_Perc <= 5 and Last_Trend > Last_Price:
            Close_Status = True

        Entry = Signals['Entry'].any() and not Signals['Entry'].all()
        Last_Entry = Signals.loc[order-1, 'Entry']
        if Entry == True and Last_Entry == True:
            Break_Status = True

        # Kırılma (Breakout) durumunu ekleyelim
        if Break_Status:
            L1 = [Hisseler[i], round(Last_Price, 2), round(Last_Trend, 2), round(Last_Perc, 2), Close_Status, Break_Status]
            print(L1)
            df_down_trend.loc[len(df_down_trend)] = L1
    except Exception as e:
        print(f"Error processing {Hisseler[i]}: {e}")
        pass

# Kapanışa yakın ve kırılma yaşayan hisseler
df_close = df_down_trend[(df_down_trend['Yakınlık Durumu'] == True)]
df_Breakout = df_down_trend[(df_down_trend['Kırılma Durumu'] == True)]

# Sonuçları yazdırma
print('Yakın Olan Hisseler')
print(df_close.to_string())
print('Kırılma Gerçekleşmiş Olanlar')
print(df_Breakout.to_string())

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-yjaoye5n
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-yjaoye5n
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=6f37b60e7977a46aeda06b927ccc7c1b6afae7f0ffa5a8dbbe1623d53579d61b
  Stored in directory: /tmp/pip-ephem-wheel-cache-s7l8_gak/wheels/0a/ba/99/b27476fd1e4caf0dd70445cdc6798195d3b90005a1501a12f7
Successfully built tvdatafeed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00



Lütfen bir zaman dilimi seçin (rakam girilecek):
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Zaman dilimini seçin (1-9): 7
Error processing AKBNK: 'trend'
['AKCNS', 2.06, 1.86, -10.62, False, True]
Error processing AKFIS: index 0 is out of bounds for axis 0 with size 0
['AKSEN', 0.42, 0.42, -0.2, False, True]
['ALCTL', 1.37, 1.27, -7.66, False, True]
Error processing ANSGR: 'trend'
Error processing ARMGD: 'trend'
['ASTOR', 1.27, 1.25, -1.45, False, True]
['BAYRK', 0.2, 0.17, -19.63, False, True]
['BINBN', 2.19, 1.83, -19.83, False, True]
Error processing BRKO: 'trend'
['BYDNR', 0.23, 0.22, -2.67, False, True]
['CWENE', 0.23, 0.23, -2.58, False, True]
Error processing DAGI: 'trend'
Error processing DCTTR: 'trend'
Error processing DERHL: 'trend'
['DITAS', 0.17, 0.16, -7.43, False, True]
Error processing DNISI: 'trend'
['DOCO', 69.94, 68.43, -2.21, False, True]
['DOFER', 0.28, 0.27, -4.93, False, True]
Error processing DYOBY: 'trend

ERROR:tvDatafeed.main:Connection to remote host was lost.
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


Error processing MEGAP: 'NoneType' object has no attribute 'reset_index'
['MEGMT', 0.33, 0.32, -4.03, False, True]
Error processing MGROS: 'trend'
['MMCAS', 0.26, 0.25, -3.5, False, True]
['MOGAN', 0.13, 0.13, -2.92, False, True]
['OBASE', 0.34, 0.33, -3.45, False, True]
['PENGD', 0.07, 0.07, -7.14, False, True]
['PKART', 0.79, 0.73, -7.24, False, True]
Error processing PLTUR: 'trend'
Error processing RAYSG: 'trend'
['RYGYO', 0.18, 0.17, -3.5, False, True]
Error processing SAHOL: 'trend'
['SELVA', 0.14, 0.13, -5.33, False, True]
Error processing SMRVA: 'trend'
Error processing SUNTK: 'trend'
['TAVHL', 2.86, 2.82, -1.52, False, True]
['TBORG', 1.31, 1.31, -0.39, False, True]
['TCELL', 1.06, 1.02, -4.54, False, True]
Error processing TERA: 'trend'
['TMPOL', 0.83, 0.82, -0.84, False, True]
Error processing TRGYO: 'trend'
Error processing TUREX: 'trend'
Error processing TURSG: 'trend'
Error processing UFUK: 'trend'
Error processing ULUFA: 'trend'
['ULUSE', 1.72, 1.61, -6.34, False, True]
E